# Hardware resources and Julia threads

## Hardware resources

When parallelizing your code it is important to know what hardware resources are
available so you can parallelize accordingly.  Two of the most important
resources in this context are: the total number of CPUs and the total amount of
memory.  This notebook covers these topics and serves as a prerequisite for the
other notebooks.

### CPU count

The number of CPUs can be far higher than the number of physical CPU
chips that the computer has.  Modern CPU chips typically contain multiple *CPU
cores* on their silicon die.  On top of that, many CPU cores also support
*simultaneous multithreading* (SMT) which, as the name implies, allows a single
CPU core to process multiple (typically two) threads of execution
simultaneously.

Julia provides the total number of CPUs available in `Sys.CPU_THREADS`.
Executing the cell below will show how many CPUs are available on your system.

In [1]:
Sys.CPU_THREADS

32

### Total memory

Code running in parallel often require additional memory (e.g. when loading
multiple data files in parallel).  It is important to be aware of the total
memory available on the system to avoid out-of-memory problems.  These can
manifest as `OutOfMemoryError`s in Julia or process crashes or an extreme slow
down if the system starts using swap space.  Knowing the total amount of
available memory is only part of the picture.  You also need to know/understand
the memory requirements of your code so that you can avoid over-parallelizing.
For example, if each parallel path requires `X GiB` of memory and the system has
`NX GiB` of memory, you will likely run into problems if you run `>N` parallel
paths because then the total memory requirement will exceed the system's memory.

Julia provides `Sys.total_memory()` to obtain the total memory available to the
process.  Note that this may differ from the physical memory of the system due
to constraints imposed by the system administrators.  Julia also provides
`Sys.total_physical_memory` to obtain the total unconstrained size of the
system's memory. Executing the cell below will show how both of these values
for your system.  Because these functions return a large value, we use Julia's
`Base.format_bytes` function to display a more human friendly representation.

In [2]:
Base.format_bytes(Sys.total_memory()), Base.format_bytes(Sys.total_physical_memory())

("503.661 GiB", "503.661 GiB")

## Julia threads

### Thread count

Julia creates its threads at startup.  By default, Julia v1.10 and earlier only
create a single thread at startup.  Threads cannot be added at runtime, so to
use multiple threads you have to tell Julia at startup time how many threads you
want to have available using either the `JULIA_NUM_THREADS` environment variable
or the `--threads` (or its `-t` equivalent) command line option.  The latter
overrides the former.  The number of threads may be given as an integer or, on
modern versions of Julia, the word `auto` which will create `Sys.CPU_THREADS`
threads.

When running a Julia notebook, the Julia process is a notebook kernel which is
usually not started from the command line.  In this case you will have to find
out how command line options can be specified for your Jupyter environment.  For
example, you may be able to setup a user kernelspec.  If running the notebook
using Visual Studio Code with the Julia extension, you can specify the
`NumTreads` value in the Julia extension settings.

The total number of Julia threads can be obtained by calling
`Threads.nthreads()`.  Executing the cell below will show this value.  If you
see `1` then your notebook's kernel is running with just a single thread and the
multi-threading code presented in future notebooks will only run on that single
thread, which is not what we want for parallelization!  If you see `1`, please
reconfigure Jupyter to start your kernel with multiple threads.

In [3]:
Threads.nthreads()

32

### Utilizing threads

The threads that Julia creates are general purpose worker threads available for
our use.  Some packages (e.g. `BLAS` and `FFTW`) use libraries that can create
threads internally, independent from the Julia threads.  These threads remain
self-contained in the library that manages them.  These notebooks don't use such
pacakges, but if your project does then you may want to be cognizant of their
thread usage when planning how many Julia worker threads to use.

We can use the worker threads that Julia creates at startup to parallelize our
code, but this does not happen automatically by merely having the threads.  We
have control over and must specify where and how we utilize these threads.  This
topic is covered in the next notebook.

## Parting thoughts

Now that we know how to get the total memory and number of threads available to
our Julia process, we can compute how much memory each thread can use on average
and still fit into system memory, ignoring for now the memory requirements of
system processes or (gasp!) other users.

In [4]:
# Compute maximum memory per thread (ignoring overheads)
Base.format_bytes(Sys.total_memory() / Threads.nthreads())

"15.739 GiB"

You are now ready to enjoy the next notebook!